In [3]:
import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [3]:
from keras_image_helper import create_preprocessor

preprocessor = create_preprocessor('resnet50', target_size=(224, 224))

In [4]:
url = "https://github.com/jg5xraydelta/SkinCancerCapstone/blob/main/data/test/malignant/1.jpg?raw=true"
X = preprocessor.from_url(url)

In [5]:
X.shape

(1, 299, 299, 3)

In [6]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [7]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'skin-cancer-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_layer_11'].CopyFrom(np_to_protobuf(X)) 

In [8]:
pb_result = stub.Predict(pb_request, timeout=20.0)

In [9]:
pred = pb_result.outputs['output_0'].float_val

In [10]:
labels = [
    'benign',
    'malignant'
] 

result = {c: p for c, p in zip(labels, pred)}

In [11]:
result

{'dress': -1.8682907819747925,
 'hat': -4.761244773864746,
 'longsleeve': -2.316983461380005,
 'outwear': -1.062570333480835,
 'pants': 9.887161254882812,
 'shirt': -2.812433958053589,
 'shoes': -3.666283130645752,
 'shorts': 3.200361728668213,
 'skirt': -2.6023383140563965,
 't-shirt': -4.835046768188477}